In [1]:
import numpy as np
import random
import doctest

def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

def cost_function(network, test_data, onehot=True):
    c = 0
    for example, y in test_data:
        if not onegot:
            y = np.eye(3,1, k=-int(y))
        yhat = network.feedforward(example)
        c += np.sum((y - yhat)**2)
    return c / len(test_data)

In [2]:
class Network:

    def __init__(self, sizes, output=True):
        """
        Список ``sizes`` содержит количество нейронов в соответствующих слоях
        нейронной сети. К примеру, если бы этот лист выглядел как [2, 3, 1],
        то мы бы получили трёхслойную нейросеть, с двумя нейронами в первом
        (входном), тремя нейронами во втором (промежуточном) и одним нейроном
        в третьем (выходном, внешнем) слое. Смещения и веса для нейронных сетей
        инициализируются случайными значениями, подчиняющимися стандартному нормальному
        распределению. Обратите внимание, что первый слой подразумевается слоем, 
        принимающим входные данные, поэтому мы не будем добавлять к нему смещение 
        (делать это не принято, поскольку смещения используются только при 
        вычислении выходных значений нейронов последующих слоёв)
        """

        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]
        self.output = output

    def feedforward(self, a):
        """
        Вычислить и вернуть выходную активацию нейронной сети
        при получении ``a`` на входе (бывшее forward_pass).
        """
        for b, w in zip(self.biases, self.weights):
            print("w: \n",w)
            print("b: \n",b)
            print("a: \n",a)
            print("z: \n", (np.dot(w, a)))
            a = sigmoid(np.dot(w, a)+b)
            print("a2: \n", a)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None):
        """
        Обучить нейронную сеть, используя алгоритм стохастического
        (mini-batch) градиентного спуска. 
        ``training_data`` - лист кортежей вида ``(x, y)``, где 
        x - вход обучающего примера, y - желаемый выход (в формате one-hot). 
        Роль остальных обязательных параметров должна быть понятна из их названия.
        Если предоставлен опциональный аргумент ``test_data``, 
        то после каждой эпохи обучения сеть будет протестирована на этих данных 
        и промежуточный результат обучения будет выведен в консоль. 
        ``test_data`` -- это список кортежей из входных данных 
        и номеров правильных классов примеров (т.е. argmax(y),
        если y -- набор ответов в той же форме, что и в тренировочных данных).
        Тестирование полезно для мониторинга процесса обучения,
        но может существенно замедлить работу программы.
        """

        if test_data is not None: n_test = len(test_data)
        n = len(training_data)
        success_tests = 0
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data is not None and self.output:
                success_tests = self.evaluate(test_data)
                print("Эпоха {0}: {1} / {2}".format(
                    j, success_tests, n_test))
            elif self.output:
                print("Эпоха {0} завершена".format(j))
        if test_data is not None:
            return success_tests / n_test

    def update_mini_batch(self, mini_batch, eta):
        """
        Обновить веса и смещения нейронной сети, сделав шаг градиентного
        спуска на основе алгоритма обратного распространения ошибки, примененного
        к одному mini batch.
        ``mini_batch`` - список кортежей вида ``(x, y)``,
        ``eta`` - величина шага (learning rate).
        """
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            
        eps = eta / len(mini_batch)
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]
        
    def backprop(self, x, y):
        """
        Возвращает кортеж ``(nabla_b, nabla_w)`` -- градиент целевой функции по всем параметрам сети.
        ``nabla_b`` и ``nabla_w`` -- послойные списки массивов ndarray,
        такие же, как self.biases и self.weights соответственно.
        """
        # Эту функцию необходимо реализовать
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        
        print("\nnabla_b: \n{0}".format(nabla_b))
        print("\nnabla_w: \n{0}".format(nabla_w))
        # прямое распространение (forward pass)

        a = [np.zeros((s,1)) for s in self.sizes]
        #print("\nx: \n",x)
        a[0] = np.array(x)
        print("\na[0]: \n", a[0].shape)
        #prevA = np.array(x).T
        #print("\nprevA: \n",prevA)

        for i, b, w in zip(range(1,len(a)),self.biases, self.weights):
            # посчитать активации
            
            print("\nw: \n",w, w.shape)
            print("\nb: \n",b, b.shape)
            a[i] = sigmoid(np.dot(w,a[i-1]) + b)
            #print("\na[{0}]: {1}\n{2}".format(i-1, a[i-1].shape, a[i-1]))
            #print("\na1[{0}]: {1}\n{2}".format(i, a[i].shape, a[i]))
        
        print("\na: \n", a)
        print("\na[-1]: \n", a[-1], type(a))
        print("\ny: \n", np.array(y), np.array(y).shape)
        
        # обратное распространение (backward pass)
        
        #delta =  np.dot((a[-1] - np.array(y).T),(a[-1]*(1-a[-1])).T)# ошибка выходного слоя
        
        delta =  (a[-1] - np.array(y)) * a[-1]*(1-a[-1])# ошибка выходного слоя
        print("\ndelta: \n",delta, type(delta), delta.shape)
        nabla_b[-1] =  delta # производная J по смещениям выходного слоя
        nabla_w[-1] =  delta.dot(a[-2].T) # производная J по весам выходного слоя

        print("\nnabla_b: \n",nabla_b, type(nabla_b))
        print("\nnabla_w: \n",nabla_w, type(nabla_w))
        
        
        # Обратите внимание, что переменная l в цикле ниже используется
        # немного иначе, чем в лекциях.  Здесь l = 1 означает последний слой, 
        # l = 2 - предпоследний и так далее.  
        # Мы перенумеровали схему, чтобы с удобством для себя 
        # использовать тот факт, что в Python к переменной типа list 
        # можно обращаться по негативному индексу.
        

        for l in range(2, self.num_layers):
            # дополнительные вычисления, чтобы легче записывалось
            #
            
            #delta =  np.dot((delta*self.weights[-l+1]),(a[-l]*(1-a[-l])))# ошибка на слое L-l
            
            delta = self.weights[-l+1].T.dot(delta) * a[-l]*(1-a[-l])
            #delta =  (self.weights[-l+1].T.dot(delta) * a[-l]*(1-a[-l])).mean(axis=1)# ошибка на слое L-l
            nabla_b[-l] =  delta# производная J по смещениям L-l-го слоя
            nabla_w[-l] =  delta.dot(a[-l-1].T)# производная J по весам L-l-го слоя
            
            print("\ndelta: \n",delta, type(delta), delta.shape)
            print("\nnabla_b: \n{0}".format(nabla_b))
            print("\nnabla_w: \n{0}".format(nabla_w))
        return nabla_b, nabla_w

    def evaluate(self, test_data):
        """
        Вернуть количество тестовых примеров, для которых нейронная сеть
        возвращает правильный ответ. Обратите внимание: подразумевается,
        что выход нейронной сети - это индекс, указывающий, какой из нейронов
        последнего слоя имеет наибольшую активацию.
        """
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """
        Возвращает вектор частных производных (\partial C_x) / (\partial a) 
        целевой функции по активациям выходного слоя.
        """
        return (output_activations-y)
    
# Описание теста. Само по себе оно ещё ничего не проверяет,
# это просто функция, которую можно вызвать.
def sample_backprop1():
    """
    Пример тестового случая. Создает сеть, запускает функцию backprop и
    проверяет возвращенные значения.
    
    >>> nabla_b, nabla_w = sample_backprop1()
    >>> print(nabla_b[0])
    [[ 0.00214254]
     [-0.05287709]]
    >>> print(nabla_w[0])
    [[ 0.00214254  0.00428509  0.00642763]
     [-0.05287709 -0.10575419 -0.15863128]]
    """
    nn = Network([3,2])
    nn.biases = [np.array([[-1], [-1]])]
    nn.weights = [np.array([[-1, 1,-1], [ 1, -1, 1]])]
    x = np.array([[1], [2], [3]])
    y = np.array([[0], [1]])
    
    return nn.backprop(x, y)

# Непосредственный запуск теста.
# Указывается имя объекта (функции, класса и т. п.), из описания которой берутся тесты.
# Обычно аргумент verbose=True не используют, и если всё хорошо,
# то система просто ничего не выводит. Мы оставили его для наглядности.
doctest.run_docstring_examples(sample_backprop1, globals(), verbose=True)

Finding tests in NoName
Trying:
    nabla_b, nabla_w = sample_backprop1()
Expecting nothing
**********************************************************************
File "__main__", line 191, in NoName
Failed example:
    nabla_b, nabla_w = sample_backprop1()
Expected nothing
Got:
    <BLANKLINE>
    nabla_b: 
    [array([[0.],
           [0.]])]
    <BLANKLINE>
    nabla_w: 
    [array([[0., 0., 0.],
           [0., 0., 0.]])]
    <BLANKLINE>
    a[0]: 
     (3, 1)
    <BLANKLINE>
    w: 
     [[-1  1 -1]
     [ 1 -1  1]] (2, 3)
    <BLANKLINE>
    b: 
     [[-1]
     [-1]] (2, 1)
    <BLANKLINE>
    a: 
     [array([[1],
           [2],
           [3]]), array([[0.04742587],
           [0.73105858]])]
    <BLANKLINE>
    a[-1]: 
     [[0.04742587]
     [0.73105858]] <class 'list'>
    <BLANKLINE>
    y: 
     [[0]
     [1]] (2, 1)
    <BLANKLINE>
    delta: 
     [[ 0.00214254]
     [-0.05287709]] <class 'numpy.ndarray'> (2, 1)
    <BLANKLINE>
    nabla_b: 
     [array([[ 0.00214254],
